<a href="https://colab.research.google.com/github/grasbiel/analise_agua/blob/main/An%C3%A1lise_Rios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Célula 1 - Importações

In [ ]:
# importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from google.colab import files
import io
from sklearn.neighbors import KNeighborsClassifier

!pip install openpyxl -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Upload do arquivo

In [ ]:
"""
print("Faça o upload do arquivo Excel:")
uploaded=files.upload()

nome_arquivo = next(iter(uploaded))


"""


'\nprint("Faça o upload do arquivo Excel:")\nuploaded=files.upload()\n\nnome_arquivo = next(iter(uploaded))\n\n\n'

Leitura do arquivo

In [ ]:
nome_arquivo = "/content/drive/MyDrive/dataset rios/Consolidado.xlsx"

In [ ]:
try:
  df_bruto = pd.read_excel(nome_arquivo)
  print(f"Total de linha {df_bruto.shape}")

except FileNotFoundError:
  print("Arquivo não encontrado")
  raise SystemExit()

except Exception as e:
  print(f"Erro ao ler arquivo {e}")
  raise SystemExit()

Total de linha (1366, 39)


In [ ]:
def padronizar_texto_simples(series):

  series = series.astype(str)
  series = series.str.replace(r"\(.*\)", "", regex= True)
  series = series.str.normalize('NFKD').str.encode('ascii', errors="ignore").str.decode('utf-8')
  series = series.str.upper()
  series = series.str.strip()

  return series



In [ ]:
def limpar_coordenadas(coord, tipo="lat"):

  # 1. Se já for número (float/int), apenas verifica o sinal
  if isinstance(coord, (int, float)):
    if pd.isna(coord): return np.nan
    valor = float(coord)

  # 2. Se for string, vamos limpar
  elif isinstance(coord, str):
    c = coord.strip()

    # Caso específico: 02°49138'' -> Tenta extrair partes numéricas
    # Remove aspas e troca grau por ponto se parecer um decimal disfarçado
    if "°" in c and "''" in c and "'" not in c:
        c = c.replace("''", "").replace("°", ".")

    # Remove caracteres que não são números, ponto ou sinal negativo
    # Mantém apenas: 0-9, . , -
    c_limpa = re.sub(r'[^0-9\.\-]', '', c.replace(',', '.'))

    try:
      valor = float(c_limpa)
    except ValueError:
      return np.nan
  else:
      return np.nan


  # Se o valor for 0, deixa 0. Se for positivo, inverte.
  if valor > 0:
      valor = -valor

  if tipo == 'lat' and (valor < -90 or valor > 90): return np.nan
  if tipo == 'lon' and (valor < -180 or valor > 180): return np.nan

  return valor

In [ ]:
coluna_municipio = "Nome Municipio"

if coluna_municipio not in df_bruto.columns:
  print(f"a coluna {coluna_municipio} não foi encontrada")

else:
  df_bruto['municipio_limpo'] = padronizar_texto_simples(df_bruto[coluna_municipio])

  df_sao_luis_filtrado = df_bruto[df_bruto['municipio_limpo'] == 'SAO LUIS'].copy()

  print(f"Total de linhas filtradas {len(df_sao_luis_filtrado)}")

  print("Nomes padronizados para SAO LUIS:")
  print(df_sao_luis_filtrado[coluna_municipio].value_counts())

  df_limpo = df_sao_luis_filtrado

Total de linhas filtradas 160
Nomes padronizados para SAO LUIS:
Nome Municipio
São Luís    144
São Luis     16
Name: count, dtype: int64


Célula 2

In [ ]:
try:
  df_sao_luis_limpo = df_limpo.copy()
  print(f"Data frame com {len(df_sao_luis_limpo)} linhas encontrado")

except NameError:
  print("Arquivo não encontrado")
  raise SystemExit()

print(f"Forma inicial dos dados {df_sao_luis_limpo.shape}")

print("Renomear colunas")
df_sao_luis_limpo = df_sao_luis_limpo.rename(columns={
    "Data da Coleta (dd/mm/aaaa)": "data_coleta",
    "Hora (hh:mm)": "hora_coleta",
    "Posição horizontal da coleta (latitude)": "latitude",
    "Posição vertical da coleta (longitude)": "longitude",
    "Choveu nas ultimas 24 horas (Sim - Não)": "choveu_24h",
    "Vazão (m3/s) / PONTO SECO?": "vazao_m3s",
    "Temperatura do ar (°C)": "temp_ar_c",
    "Temperatura da água (°C)": "temp_agua_c",
    "Oxigênio dissolvido (mg/L 02)": "oxigenio_dissolvido_mgL",
    "Condutividade Elétrica Específica (25°C) (µS/cm a 25°C)": "condutividade_eletrica_uScm",
    "Turbidez (NTU)": "turbidez_ntu",
    "Salinidade (‰)": "salinidade_ppt",
    "Alcalinidade (mg/L)": "alcalinidade_mgL",
    "Transparência da água (m)": "transparencia_agua_m",
    "Sólidos Dissolvidos (mg/L)": "solidos_dissolvidos_mgL",
    "Sólidos em suspensão (mg/L)": "solidos_suspensao_mgL",
    "Nome do Corpo D'Água": "nome_corpo_agua"
})

print("Conversão de dados")
df_sao_luis_limpo['data_coleta'] = pd.to_datetime(df_sao_luis_limpo['data_coleta'], errors='coerce')
df_sao_luis_limpo['hora_coleta'] = pd.to_timedelta(df_sao_luis_limpo['hora_coleta'].astype(str), errors="coerce")
df_sao_luis_limpo['timestamp_coleta'] = df_sao_luis_limpo['data_coleta'] + df_sao_luis_limpo['hora_coleta']

if 'choveu_24h' in df_sao_luis_limpo.columns:
  df_sao_luis_limpo["choveu_24h"] = df_sao_luis_limpo["choveu_24h"].str.lower().str.strip()
  df_sao_luis_limpo["choveu_24h"] = df_sao_luis_limpo["choveu_24h"].map({'sim': True, 'não': False})

colunas_numericas = ['vazao_m3s', 'temp_ar_c', 'temp_agua_c', 'pH',
    'oxigenio_dissolvido_mgL', 'condutividade_eletrica_uScm',
    'turbidez_ntu', 'salinidade_ppt', 'alcalinidade_mgL',
    'transparencia_agua_m', 'solidos_dissolvidos_mgL',
    'solidos_suspensao_mgL', 'Profundidade (m)'
]

valores_ausentes_num = ['-','N.A.', 'NA', 'Ponto Seco', 'PCD SEM DADO', '']


for col in colunas_numericas:
  if col in df_sao_luis_limpo.columns:

    df_sao_luis_limpo[col] = pd.to_numeric(df_sao_luis_limpo[col], errors="coerce")


col_lat = 'Posição horizontal da coleta (latitude)'
col_lon = 'Posição vertical da coleta (longitude)'

# Se os nomes forem diferentes no seu Excel, o script tenta achar
if col_lat not in df_sao_luis_limpo.columns:
    # Tenta achar colunas que contenham "latitude" no nome
    possiveis = [c for c in df_sao_luis_limpo.columns if 'latitude' in c.lower()]
    if possiveis: col_lat = possiveis[0]

if col_lon not in df_sao_luis_limpo.columns:
    possiveis = [c for c in df_sao_luis_limpo.columns if 'longitude' in c.lower()]
    if possiveis: col_lon = possiveis[0]

print(f"Usando colunas: Lat='{col_lat}', Lon='{col_lon}'")

if 'latitude' in df_sao_luis_limpo.columns:
  df_sao_luis_limpo['latitude'] = df_sao_luis_limpo['latitude'].apply(lambda x: limpar_coordenadas(x, tipo= "lat"))

if 'longitude'in df_sao_luis_limpo.columns:
  df_sao_luis_limpo['longitude'] = df_sao_luis_limpo['longitude'].apply(lambda x: limpar_coordenadas(x, tipo="lon"))


print("\n--- Verificação das Coordenadas Corrigidas (Amostra) ---")
amostra_ruim = df_sao_luis_limpo[df_sao_luis_limpo['latitude'].isna()]
if not amostra_ruim.empty:
    print(f"⚠️ Atenção: {len(amostra_ruim)} linhas ficaram sem latitude válida (NaN).")
else:
    print("✅ Todas as latitudes foram convertidas com sucesso.")

print("\nExemplo de dados corrigidos (Top 5):")
print(df_sao_luis_limpo[['latitude', 'longitude']].head(10))

colunas_para_analise = [
    'latitude', 'longitude', 'vazao_m3s', 'temp_ar_c', 'temp_agua_c', 'pH',
    'oxigenio_dissolvido_mgL', 'condutividade_eletrica_uScm', 'turbidez_ntu'
]


colunas_existentes = [col for col in colunas_para_analise if col in df_sao_luis_limpo.columns]

if colunas_existentes:
  print("\n Resumo Estatístico")
  print(df_sao_luis_limpo[colunas_existentes].describe())

Data frame com 160 linhas encontrado
Forma inicial dos dados (160, 40)
Renomear colunas
Conversão de dados
Usando colunas: Lat='latitude', Lon='longitude'

--- Verificação das Coordenadas Corrigidas (Amostra) ---
⚠️ Atenção: 36 linhas ficaram sem latitude válida (NaN).

Exemplo de dados corrigidos (Top 5):
   latitude  longitude
0  -2.66701  -44.32013
1  -2.68905  -44.21021
2  -2.53104  -44.27464
3  -2.65107  -44.21425
4  -2.56592  -44.28427
5  -2.49140  -44.25871
6  -2.52333  -44.20012
7  -2.47858  -44.21030
8  -2.52398  -44.27910
9  -2.49303  -44.25415

 Resumo Estatístico
         latitude   longitude  vazao_m3s   temp_ar_c  temp_agua_c          pH  \
count  124.000000  122.000000  24.000000  157.000000   157.000000  158.000000   
mean    -2.591032  -44.247817   2.464587   27.353618    28.799873    7.157278   
std      0.153212    0.048205   6.662844    1.872726     1.375053    1.180027   
min     -3.614790  -44.381250   0.001600   23.200000    25.790000    1.560000   
25%     -2.66

In [ ]:
print("Módulo de Reclassificação Espacial")

try:
    df_reclass = df_sao_luis_limpo.copy()
    print(f"Analisando {len(df_reclass)} registros...")
except NameError:
    print("🚨 ERRO: 'df' não encontrado.")
    raise SystemExit()

# Limpeza de Coordenadas e Nomes

def limpar_coord_float(x):
    try:
        return float(x)
    except:
        return np.nan

df_reclass['lat_ml'] = df_reclass['latitude'].apply(limpar_coord_float)
df_reclass['lon_ml'] = df_reclass['longitude'].apply(limpar_coord_float)

# Remover linhas sem coordenadas para o treino
df_validos = df_reclass.dropna(subset=['lat_ml', 'lon_ml']).copy()

# Padronizar nomes (Upper case) para evitar que "Rio Anil" e "RIO ANIL" sejam considerados diferentes
coluna_rio = "nome_corpo_agua"
df_validos['rio_original'] = df_validos[coluna_rio].astype(str).str.upper().str.strip()

# Ignorar nomes genéricos ou vazios no treino
nomes_invalidos = ['NAN', 'N.A.', '0', 'UNDEFINED', '']
df_treino = df_validos[~df_validos['rio_original'].isin(nomes_invalidos)]

print(f"Base de conhecimento: {len(df_treino)} pontos com nomes e coordenadas válidas.")

# --- 3. O Algoritmo de Correção (Leave-One-Out Logic) ---
# Usaremos n_neighbors=1, mas aplicaremos uma lógica manual para não pegar o próprio ponto

X = df_treino[['lat_ml', 'lon_ml']].values
y = df_treino['rio_original'].values

# Criamos o modelo
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)

# Listas para armazenar resultados
novos_nomes = []
alteracoes = 0

print("\nVerificando consistência espacial...")

# Iterar sobre CADA linha para prever seu nome baseado nos OUTROS
for index, row in df_reclass.iterrows():

    # Se não tem coordenada, mantém o original
    if pd.isna(row['lat_ml']) or pd.isna(row['lon_ml']):
        novos_nomes.append(row[coluna_rio])
        continue

    # Coordenada do ponto atual
    ponto_atual = [[row['lat_ml'], row['lon_ml']]]

    # Encontrar os 2 vizinhos mais próximos
    # (O 1º vizinho será ele mesmo se ele estiver na base de treino, então pegamos o 2º)
    distancias, indices = knn.kneighbors(ponto_atual, n_neighbors=2)

    # Lógica:
    # O kneighbors retorna indices baseados no df_treino (matriz X)
    vizinho_idx = indices[0][0] # Pega o mais próximo
    distancia = distancias[0][0]

    # Se a distância for 0 (é ele mesmo), pegamos o segundo vizinho
    if distancia == 0.0 and len(indices[0]) > 1:
        vizinho_idx = indices[0][1]
        distancia = distancias[0][1]

    # Nome sugerido pelo vizinho
    nome_sugerido = y[vizinho_idx]
    nome_original = str(row[coluna_rio]).upper().strip()

    # Adicionar à lista
    novos_nomes.append(nome_sugerido)

    # Verificar se houve mudança (apenas para log)
    if nome_sugerido != nome_original and nome_original not in nomes_invalidos:
        # print(f"Alteração ID {index}: {nome_original} -> {nome_sugerido} (Dist: {distancia:.5f})")
        alteracoes += 1

# Aplicar Alterações

# Criar coluna nova para comparação
df_reclass['Rio_Sugerido_ML'] = novos_nomes

# Criar coluna de Flag (Verdadeiro se o nome foi alterado)
df_reclass['Nome_Foi_Alterado'] = df_reclass.apply(
    lambda x: str(x[coluna_rio]).upper().strip() != str(x['Rio_Sugerido_ML']), axis=1
)

# Atualizar o DataFrame principal
df_reclass['Rio_Original_Backup'] = df_reclass[coluna_rio]
df_reclass[coluna_rio] = df_reclass['Rio_Sugerido_ML']

# Remover colunas auxiliares de ML
df_final_ml = df_reclass.drop(columns=['lat_ml', 'lon_ml', 'rio_original'], errors='ignore')

# Relatório e Salvamento

print("\n" + "="*40)
print(f"RELATÓRIO DE CORREÇÃO ESPACIAL")
print("="*40)
print(f"Total de registros analisados: {len(df_reclass)}")
print(f"Total de reclassificações sugeridas: {alteracoes}")
print("-" * 40)

# Mostrar exemplos das alterações (se houver)
if alteracoes > 0:
    print("Exemplos de correções realizadas:")
    cols_show = ['Rio_Original_Backup', 'Rio_Sugerido_ML', 'latitude', 'longitude']
    print(df_final_ml[df_final_ml['Nome_Foi_Alterado'] == True][cols_show].head(10))
else:
    print("Nenhuma inconsistência espacial encontrada. Os nomes atuais parecem corretos.")

# Atualizar a variável global para as próximas células
df_sao_luis_limpo = df_final_ml


Módulo de Reclassificação Espacial
Analisando 160 registros...
Base de conhecimento: 94 pontos com nomes e coordenadas válidas.

Verificando consistência espacial...

RELATÓRIO DE CORREÇÃO ESPACIAL
Total de registros analisados: 160
Total de reclassificações sugeridas: 2
----------------------------------------
Exemplos de correções realizadas:
    Rio_Original_Backup    Rio_Sugerido_ML  latitude  longitude
490          Rio Tibiri      RIO ITAPIRACÓ -2.518590 -44.070910
722   Rio dos Cachorros  Rio dos Cachorros -2.667010        NaN
723          Rio Tibiri         Rio Tibiri -2.688950        NaN
726            Rio Anil        RIO BACANGA -3.614790 -44.381250
932                 NaN         RIO CALHAU -2.491270 -44.259100
933                 NaN                NaN       NaN        NaN
934                 NaN      RIO JAGUAREMA -2.478360 -44.210420
935                 NaN                NaN       NaN        NaN
942                 NaN           RIO ANIL -2.523875 -44.279067
943          

In [ ]:
import folium
from folium.plugins import MarkerCluster
from sklearn.neighbors import NearestNeighbors
import matplotlib.colors as mcolors

try:
    df_val = df_final_ml.copy()
    print(f"Validando DataFrame com {len(df_val)} registros.")
except NameError:
    print("🚨 ERRO: 'df_limpo' não encontrado. Execute as etapas anteriores primeiro.")
    raise SystemExit()

coluna_rio = "nome_corpo_agua"

# --- ETAPA 1: Verificação Quantitativa (Sobrou alguém vazio?) ---
print("\n1. VERIFICAÇÃO DE PREENCHIMENTO")
vazios = df_val[coluna_rio].isna().sum()
nulos_texto = df_val[df_val[coluna_rio].astype(str).str.upper().isin(['NAN', 'N.A.', '0', ''])].shape[0]
total_pendente = vazios + nulos_texto

if total_pendente == 0:
    print("✅ Sucesso: Todos os registros possuem um nome de rio preenchido!")
else:
    print(f"⚠️ Atenção: Ainda existem {total_pendente} registros sem nome de rio válido.")
    print("Isso acontece se o registro não tinha coordenadas para o ML calcular.")

# --- ETAPA 2: Verificação de Confiança (Distância do Vizinho) ---
print("\n2. VERIFICAÇÃO DE CONFIANÇA (Distância)")
# Vamos calcular quão longe estava o vizinho usado para classificar
# Se for muito longe, pode ser um erro.

df_coords = df_val.dropna(subset=['latitude', 'longitude'])
X = df_coords[['latitude', 'longitude']].values

# Achar o vizinho mais próximo de cada ponto
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

# A distância para o vizinho mais próximo (o índice 1, pois o 0 é ele mesmo)
# Nota: A distância aqui é em graus decimais (aproximado).
# 0.01 grau ~= 1.1km. 0.001 ~= 110 metros.
df_coords['distancia_vizinho'] = distances[:, 1]

# Definir um limiar de alerta (ex: vizinho a mais de ~1km de distância)
limiar_alerta = 0.01
suspeitos = df_coords[df_coords['distancia_vizinho'] > limiar_alerta]

if not suspeitos.empty:
    print(f"⚠️ Alerta: {len(suspeitos)} pontos estão muito isolados (>1km do vizinho mais próximo).")
    print("A classificação destes pontos pode não ser confiável.")
    print(suspeitos[['Nome Municipio', coluna_rio, 'latitude', 'longitude']].head())
else:
    print("✅ Consistência: Todos os pontos têm vizinhos próximos (<1km), indicando boa densidade.")


print("\n3. GERANDO MAPA DE COERÊNCIA ESPACIAL...")
print("Observe o mapa: Pontos do mesmo rio devem formar aglomerados da mesma cor.")

# Função para gerar cores únicas para cada nome de rio (Hash de string para cor)
def string_to_color(string):
    if pd.isna(string): return 'black'
    hash_val = hash(string)
    # Lista de cores legíveis do matplotlib (excluindo brancos/claros demais)
    cores_disponiveis = list(mcolors.TABLEAU_COLORS.values()) + ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue']
    return cores_disponiveis[hash_val % len(cores_disponiveis)]

# Centro do mapa
centro = [df_coords['latitude'].mean(), df_coords['longitude'].mean()]
mapa_validacao = folium.Map(location=centro, zoom_start=11, tiles='CartoDB positron') # Fundo limpo

# Adicionar pontos
for _, row in df_coords.iterrows():
    rio_nome = str(row[coluna_rio]).upper()
    cor = string_to_color(rio_nome)

    # Se for um ponto suspeito (isolado), destacamos com um círculo maior
    raio = 5
    if row['distancia_vizinho'] > limiar_alerta:
        raio = 10 # Ponto maior para chamar atenção
        popup_extra = "<br>⚠️ <b>PONTO ISOLADO (Baixa Confiança)</b>"
    else:
        popup_extra = ""

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=raio,
        color=cor,
        fill=True,
        fill_color=cor,
        fill_opacity=0.7,
        popup=folium.Popup(f"<b>Rio:</b> {rio_nome}{popup_extra}", max_width=200),
        tooltip=rio_nome
    ).add_to(mapa_validacao)

# Legenda improvisada (Top 10 rios)
top_rios = df_coords[coluna_rio].value_counts().head(10).index.tolist()
legenda_html = '''
     <div style="position: fixed;
     bottom: 50px; left: 50px; width: 180px; height: auto;
     border:2px solid grey; z-index:9999; font-size:12px;
     background-color:white; opacity:0.9; padding: 10px;">
     <b>Top Rios (Cor):</b><br>
'''
for rio in top_rios:
    cor = string_to_color(str(rio).upper())
    legenda_html += f'<i class="fa fa-circle" style="color:{cor}"></i> {rio}<br>'
legenda_html += "</div>"
mapa_validacao.get_root().html.add_child(folium.Element(legenda_html))

display(mapa_validacao)

Validando DataFrame com 160 registros.

1. VERIFICAÇÃO DE PREENCHIMENTO
⚠️ Atenção: Ainda existem 36 registros sem nome de rio válido.
Isso acontece se o registro não tinha coordenadas para o ML calcular.

2. VERIFICAÇÃO DE CONFIANÇA (Distância)
⚠️ Alerta: 1 pontos estão muito isolados (>1km do vizinho mais próximo).
A classificação destes pontos pode não ser confiável.
    Nome Municipio nome_corpo_agua  latitude  longitude
490       São Luís   RIO ITAPIRACO  -2.51859  -44.07091

3. GERANDO MAPA DE COERÊNCIA ESPACIAL...
Observe o mapa: Pontos do mesmo rio devem formar aglomerados da mesma cor.


/tmp/ipython-input-145481342.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coords['distancia_vizinho'] = distances[:, 1]


In [ ]:
#Análise Categórica
top_10_rios = None
if 'nome_corpo_agua'in df_sao_luis_limpo.columns:
  print("Contagem de Corpos Dagua (São Luís)")


  df_sao_luis_limpo['nome_corpo_agua'] = padronizar_texto_simples(df_sao_luis_limpo['nome_corpo_agua'])
  top_10_rios = df_sao_luis_limpo['nome_corpo_agua'].value_counts().nlargest(10)
  print(top_10_rios)


Contagem de Corpos Dagua (São Luís)
nome_corpo_agua
NAN                  36
RIO TIBIRI           31
RIO BACANGA          16
RIO DOS CACHORROS    16
RIO JAGUAREMA        16
RIO ITAPIRACO        16
RIO CALHAU           15
RIO ANIL             14
Name: count, dtype: int64


In [ ]:
# Gerando Gráficos Rios de São Luís

if 'pH' in df_sao_luis_limpo.columns:
  plt.figure(figsize=(10,6))
  sns.histplot(df_sao_luis_limpo['pH'].dropna(), kde=True,bins=25, color='royalblue')
  plt.axvline(6.0, color='red', linestyle='--', label="Limite Ácido (CONAMA 6.0)")
  plt.axvline(9.0, color='red', linestyle='--', label="Limie Básico (CONAMA 9.0)")
  plt.title("Distribuição de pH das Amostras em São Luís", fontsize=16)
  plt.xlabel("pH", fontsize=12);
  plt.ylabel("Contagem", fontsize=12)
  plt.legend()
  plt.show()

In [ ]:
if "turbidez_ntu" in df_sao_luis_limpo.columns and df_sao_luis_limpo['turbidez_ntu'].notna().any():
  plt.figure(figsize=(10,6))
  sns.histplot(df_sao_luis_limpo['turbidez_ntu'].dropna(), bins=50, color='brown')
  plt.title("Distribuição da Turbidez (NTU) em São Luís - Escala Log", fontsize=16)
  plt.xlabel("Turbidez (NTU)", fontsize=12)
  plt.ylabel("Contagem", fontsize=12)
  plt.yscale("log")
  plt.show()

In [ ]:
if colunas_existentes:
  plt.figure(figsize=(14,10))
  corr_matrix_focada = df_sao_luis_limpo[colunas_existentes].corr()
  sns.heatmap(corr_matrix_focada, annot= True, fmt='.2f', cmap="coolwarm", linewidths=0.5)
  plt.title("Mapa de Calor de Correlação (São Luís)", fontsize=16)
  plt.xticks(rotation=45, ha="right")
  plt.yticks(rotation=0)
  plt.tight_layout()
  plt.show()

Célula 3 - Classificação

In [ ]:
print("Classificação de Água")

# Definição dos Padrões (CONAMA 357/2005 - Água Doce - CLASSE 2)
CONAMA_CLASSE_2 = {
    'ph': {'min': 6.0, 'max': 9.0},
    'oxigenio_dissolvido_mgL': {'min': 5.0, 'max': np.inf}, # Não inferior a 5 mg/L
    'turbidez_ntu': {'min': -np.inf, 'max': 100.0} # Até 100 NTU
}
print("Usando padrões CONAMA 357/2005 (Água Doce, Classe 2)")

# Funções de Classificação
def classificar_parametro(valor, parametro_nome):
    if pd.isna(valor):
        return 'Sem Dado'
    limites = CONAMA_CLASSE_2.get(parametro_nome)
    if not limites:
        return 'Sem Padrão'
    if valor >= limites['min'] and valor <= limites['max']:
        return 'Dentro do Padrão'
    else:
        return 'Fora do Padrão'


# Aplicar a Classificação
print("Aplicando classificações CONAMA...")
df_sao_luis_limpo['classificacao_ph'] = df_sao_luis_limpo['pH'].apply(classificar_parametro, args=('ph',))
df_sao_luis_limpo['classificacao_od'] = df_sao_luis_limpo['oxigenio_dissolvido_mgL'].apply(classificar_parametro, args=('oxigenio_dissolvido_mgL',))
df_sao_luis_limpo['classificacao_turbidez'] = df_sao_luis_limpo['turbidez_ntu'].apply(classificar_parametro, args=('turbidez_ntu',))

# Calcular Índice de Conformidade
print("Calculando índice de conformidade...")
df_sao_luis_limpo['fora_padrao_ph'] = (df_sao_luis_limpo['classificacao_ph'] == 'Fora do Padrão').astype(int)
df_sao_luis_limpo['fora_padrao_od'] = (df_sao_luis_limpo['classificacao_od'] == 'Fora do Padrão').astype(int)
df_sao_luis_limpo['fora_padrao_turbidez'] = (df_sao_luis_limpo['classificacao_turbidez'] == 'Fora do Padrão').astype(int)

df_sao_luis_limpo['total_parametros_fora'] = (
    df_sao_luis_limpo['fora_padrao_ph'] +
    df_sao_luis_limpo['fora_padrao_od'] +
    df_sao_luis_limpo['fora_padrao_turbidez']
)

# Exibir Resumo da Classificação
print("\nResumo da Classificação (São Luís)")
print("\nConformidade do pH:")
print(df_sao_luis_limpo['classificacao_ph'].value_counts(dropna=False))
print("\nConformidade do Oxigênio Dissolvido (OD):")
print(df_sao_luis_limpo['classificacao_od'].value_counts(dropna=False))
print("\nConformidade da Turbidez:")
print(df_sao_luis_limpo['classificacao_turbidez'].value_counts(dropna=False))

print("\nGerando gráficos de conformidade...")

# Gráfico de Conformidade por Parâmetro
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)
fig.suptitle('Percentual de Conformidade por Parâmetro (São Luís)', fontsize=16)

# Gráfico pH
df_ph_counts = df_sao_luis_limpo['classificacao_ph'].value_counts(normalize=True).mul(100).rename('percentual').reset_index()
sns.barplot(
    ax=axes[0],
    x='classificacao_ph',
    y='percentual',
    data=df_ph_counts,
    hue= "classificacao_ph",
    palette={'Dentro do Padrão': 'green', 'Fora do Padrão': 'red', 'Sem Dado': 'gray'}
  )

axes[0].set_title('pH (Padrão: 6.0-9.0)')
axes[0].set_ylabel('Percentual (%)'); axes[0].set_xlabel('')

# Gráfico OD
df_od_counts = df_sao_luis_limpo['classificacao_od'].value_counts(normalize=True).mul(100).rename('percentual').reset_index()
sns.barplot(
    ax=axes[1],
    x='classificacao_od',
    y='percentual',
    data=df_od_counts,
    hue='classificacao_od',
    palette={'Dentro do Padrão': 'green', 'Fora do Padrão': 'red', 'Sem Dado': 'gray'}
  )
axes[1].set_title('Oxigênio Dissolvido (Padrão: >= 5 mg/L)')
axes[1].set_ylabel(''); axes[1].set_xlabel('')

# Gráfico Turbidez
df_turb_counts = df_sao_luis_limpo['classificacao_turbidez'].value_counts(normalize=True).mul(100).rename('percentual').reset_index()
sns.barplot(
    ax=axes[2],
    x='classificacao_turbidez',
    y='percentual',
    data=df_turb_counts,
    hue='classificacao_turbidez',
    palette={'Dentro do Padrão': 'green', 'Fora do Padrão': 'red', 'Sem Dado': 'gray'}
  )
axes[2].set_title('Turbidez (Padrão: <= 100 NTU)')
axes[2].set_ylabel(''); axes[2].set_xlabel('')
plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()

# Gráfico do Índice de Conformidade
plt.figure(figsize=(10, 6))

df_sao_luis_limpo['total_parametros_fora'] = df_sao_luis_limpo['total_parametros_fora'].astype(int)

sns.countplot(
    x='total_parametros_fora',
    data=df_sao_luis_limpo,
    hue='total_parametros_fora',
    palette={0: 'darkgreen', 1: 'orange', 2: 'darkred', 3: 'black'}
)
plt.title('Distribuição do Índice de Conformidade (São Luís)', fontsize=16)
plt.xlabel('Número de Parâmetros Fora do Padrão CONAMA')
plt.ylabel('Contagem de Coletas')
plt.show()

Célula 4 - Geração de Mapas

In [ ]:
import folium
from folium.plugins import MarkerCluster
from IPython.display import display
import warnings

warnings.filterwarnings('ignore')

# Preparação dos Dados para o Mapa
df_sao_luis_limpo = df_sao_luis_limpo.dropna(subset=['latitude', 'longitude'])
print(f"Total de registros classificados com geolocalização: {df_sao_luis_limpo.shape[0]}")

if df_sao_luis_limpo.empty:
    print(f"Nenhum dado de geolocalização válido encontrado. Não é possível criar o mapa.")
    raise SystemExit()

# Calcular o centro do mapa
centro_mapa = [df_sao_luis_limpo['latitude'].mean(), df_sao_luis_limpo['longitude'].mean()]

# Função Auxiliar para Cor do Marcador (por Conformidade)
def get_marker_color_by_conformity(total_fora):
    if pd.isna(total_fora):
        return 'gray'
    total_fora = int(total_fora)
    if total_fora == 0:
        return 'green'   # 0 problemas: Em conformidade
    elif total_fora == 1:
        return 'orange'  # 1 problema: Atenção
    elif total_fora == 2:
        return 'red'     # 2 problemas: Ruim
    else: # 3 ou mais problemas
        return 'darkred' # 3 problemas: Muito Ruim

# Geração do Mapa de Conformidade (Clusters)
print("\nGerando Mapa de Conformidade (Colorido por nº de problemas CONAMA)...")

mapa_conformidade = folium.Map(location=centro_mapa, zoom_start=12, tiles='OpenStreetMap')
marker_cluster = MarkerCluster().add_to(mapa_conformidade)

for idx, row in df_sao_luis_limpo.iterrows():

    data_val = pd.to_datetime(row['timestamp_coleta']).strftime('%Y-%m-%d %H:%M') if pd.notna(row['timestamp_coleta']) else "Sem Data"

    # Monta o popup com as informações de classificação
    popup_html = f"""
    <h4><i class="fa fa-tint"></i> Detalhes da Coleta</h4>
    <b>Corpo d'Água:</b> {row["nome_corpo_agua"]}<br>
    <b>Data:</b> {data_val}<br>
    <hr>
    <h4><i class="fa fa-check-circle"></i> Status CONAMA (Classe 2)</h4>
    <b>Total de Problemas:</b> <strong>{int(row['total_parametros_fora'])}</strong><br>
    <br>
    <b>pH:</b> {row['classificacao_ph']} <em>(Valor: {row['pH']:.2f})</em><br>
    <b>Oxigênio (OD):</b> {row['classificacao_od']} <em>(Valor: {row['oxigenio_dissolvido_mgL']:.2f})</em><br>
    <b>Turbidez:</b> {row['classificacao_turbidez']} <em>(Valor: {row['turbidez_ntu']:.2f})</em>
    """

    cor_do_pino = get_marker_color_by_conformity(row['total_parametros_fora'])

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=350),
        tooltip=f"Problemas: {int(row['total_parametros_fora'])} | {row["nome_corpo_agua"]}",
        icon=folium.Icon(color=cor_do_pino, icon='tint', prefix='fa')
    ).add_to(marker_cluster)

# Exibição do mapa
print("Mapa de Conformidade - São Luís!")
display(mapa_conformidade)

In [ ]:
registros_sem_coord = df_sao_luis_limpo[df_sao_luis_limpo['latitude'].isna() | df_sao_luis_limpo['longitude'].isna()]

print(f"Total de registros sem coordenadas: {len(registros_sem_coord)}")

Célula 5 - Análise Temporal

In [ ]:
# Impacto da chuva

try:
  df_temporal = df_sao_luis_limpo.copy()
  print(f"Utilizando o dataframe de São Luís. Tamanho: {len(df_temporal)} linhas\n")

except NameError:
  print("Erro: DataFrame 'sao_luis_limpo' não encontrado")
  raise SystemExit()

df_temporal['timestamp_coleta'] = pd.to_datetime(df_sao_luis_limpo['timestamp_coleta'],errors="coerce")


# Limpando as linhas de chuva que estiverem vazias
df_chuva = df_temporal.dropna(subset=['choveu_24h','turbidez_ntu', 'total_parametros_fora']).copy()


if df_chuva.empty:
  print("Não há dados completos (com choveu 24 e dados de qualidade) para esta análise")

else:
  df_chuva['choveu_status'] = df_chuva['choveu_24h'].map({True: 'Sim (Choveu)', False: 'Não (Sem Chuva)'})

  # Gráfico 1: Turbides vs Chuva
  plt.figure(figsize=(10,6))
  sns.barplot(
      x="choveu_status",
      y="turbidez_ntu",
      data=df_chuva,
      palette={'Sim (Choveu)': '#3498db', 'Não (Sem Chuva)': '#bdc3c7'}
  )
  plt.title("Média de turbidez (NTU) vs Chuva (São Luís)", fontsize=16)
  plt.ylabel("Turbidez Média (NTU)")
  plt.xlabel("Choevu nas últimas 24h")
  plt.show()

  # Gráfico 2: Problemas CONAMA vs Chuva
  plt.figure(figsize=(10,6))
  sns.barplot(
      x= "choveu_status",
      y= "total_parametros_fora",
      data= df_chuva,
      palette= {'Sim (Choveu)': '#e74c3c', 'Não (Sem Chuva)': '#2ecc71'}
  )
  plt.title("Média de Problemas CONAMA vs. Chuva (São Luís)", fontsize=16)
  plt.ylabel("Média de Parâmetros Fora do Padrão")
  plt.show()




In [ ]:
print("Análise Série Temporal da Qualidade ao Longo do Tempo")

# Usando linhas válidas
df_serie = df_temporal.dropna(subset=['timestamp_coleta']).set_index("timestamp_coleta")

# Agrupando os dados por mês e calculando a média
df_mensal = df_serie[['turbidez_ntu', 'total_parametros_fora']].resample('M').mean()

# Removendo meses sem medição
df_mensal = df_mensal.dropna(how="all")

if df_mensal.empty:
  print("Não há dados de data suficientes")

else:
  # Gráfico 3: Turbidez ao Longo do Tempo
  plt.figure(figsize=(14,7))
  df_mensal['turbidez_ntu'].plot(linestyle='-', marker="o", color="brown")
  plt.title("Média Mensal da Turbidez NTU em São Luís", fontsize=16)
  plt.ylabel("Turbidez Média (NTU)")
  plt.xlabel("Data")
  plt.grid(True, which='both', linestyle="--", linewidth=0.5)
  plt.show()

  # Gráfico 4: Problemas CONAMA ao longo do tempo
  plt.figure(figsize=(14,7))
  df_mensal["total_parametros_fora"].plot(linestyle="-", marker="o", color="red")
  plt.title("Média Mensal de Problemas CONAMA em São Luís", fontsize=16)
  plt.ylabel("Média de Parâmetros Fora do Padrão")
  plt.xlabel("Data")
  plt.grid(True, which="both", linestyle="--", linewidth=0.5)
  plt.show()

Célula 6 - Análise Comparativa

In [ ]:
try:
  df_comparativo = df_sao_luis_limpo.copy()
  print(f"Dados carregados. Registros: {len(df_comparativo)}")

except NameError:
  print("Dados não foram carregados")
  raise SystemExit()

contagem_locais = df_comparativo['nome_corpo_agua'].value_counts()
locais_relevantes = contagem_locais[contagem_locais > 1].index

df_comp_filtrado = df_comparativo[
    (df_comparativo['nome_corpo_agua'].isin(locais_relevantes)) &
    (df_comparativo['turbidez_ntu'].notna())

]

print(f"Analisando os seguintes corpos d'agua: {list(locais_relevantes)}")


sns.set_style("whitegrid")

# Gráfico 1: Comparação de pH por Rio
plt.figure(figsize=(14,8))
sns.boxplot(x="nome_corpo_agua", y="pH", data=df_comp_filtrado, palette="Blues")
plt.axhline(y=6.0, color="r", linestyle="--", label="Mínimo CONAMA (6.0)")
plt.axhline(y=9.0, color="r", linestyle="--", label="Máximo CONAMA (9.0)")
plt.xticks(rotation=45, ha="right")
plt.title("Comparação de ph por Corpo D\'água em São Luís", fontsize=16)
plt.legend()
plt.tight_layout()
plt.show()

# Gráfico 2: Comparação de Turbidez por Rio
plt.figure(figsize=(14, 8))
sns.boxplot(x='nome_corpo_agua', y='turbidez_ntu', data=df_comp_filtrado, palette="Oranges")
plt.axhline(y=100.0, color='r', linestyle='--', label='Máximo CONAMA (100 NTU)')
plt.yscale('log') # IMPORTANTE: Escala logarítmica ajuda a ver dados muito díspares
plt.xticks(rotation=45, ha='right')
plt.title('Comparação de Turbidez por Corpo D\'água (Escala Log)', fontsize=16)
plt.ylabel('Turbidez (NTU) - Escala Log')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
colunas_existentes = df_sao_luis_limpo.columns
cols_interesse = []

termos_busca = ['fosfato', 'nitrato', 'sulfato', 'ortofosfato']

print ("Procurando dados químicos...")
for col in colunas_existentes:
  for termo in termos_busca:
    if termo in col.lower():
      cols_interesse.append(col)


cols_interesse = list(set(cols_interesse))

for col in cols_interesse:
  if df_sao_luis_limpo[col].dtype == "object":
    df_sao_luis_limpo[col] = df_sao_luis_limpo[col].astype(str).str.replace(',','.')

  df_sao_luis_limpo[col] = pd.to_numeric(df_sao_luis_limpo[col], errors="coerce")

if not cols_interesse:
  print("Não foram encontrados colunas")

else:
  print(f"Colunas enconrtadas: {cols_interesse}")

  df_nutrientes = df_sao_luis_limpo.dropna(subset=cols_interesse, how="all")

  for col_nutriente in cols_interesse:
    plt.figure(figsize=(12,6))

    contagem = df_nutrientes['nome_corpo_agua'].value_counts()
    locais_validos = contagem[contagem > 1].index
    dados_plot = df_nutrientes[df_nutrientes['nome_corpo_agua'].isin(locais_validos)].copy()

    sns.boxplot(x="nome_corpo_agua", y=col_nutriente, data=dados_plot, palette="Reds")

    plt.title(f"Concentração de {col_nutriente} por local", fontsize=14)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel(f"{col_nutriente}")
    plt.tight_layout()
    plt.show()

    max_val = df_nutrientes[col_nutriente].max()
    print(f"Máximo de {col_nutriente}: {max_val}")